# Lab 3: AgentCore Gateway로 도구를 에이전트에 안전하게 연결

## 개요

이 랩에서는 Amazon Bedrock Gateway를 사용하여 조직에서 사용 가능한 도구를 이커머스 고객 지원 에이전트와 통합하는 방법을 배웁니다.

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro)는 애플리케이션이 대형 언어 모델(LLM)에 도구와 컨텍스트를 제공하는 방법을 표준화하는 개방형 프로토콜입니다.

[Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)를 사용하면 개발자는 몇 줄의 코드만으로 API, Lambda 함수 및 기존 서비스를 MCP 호환 도구로 변환하고 Gateway 엔드포인트를 통해 에이전트에서 사용할 수 있습니다.

**워크숍 여정:**

- **Lab 1 (완료):** 에이전트 프로토타입 생성 - 기능적인 이커머스 고객 지원 에이전트 구축
- **Lab 2 (완료):** 메모리로 강화 - 대화 컨텍스트 및 개인화 추가
- **Lab 3 (현재):** Gateway & Identity로 확장 - 에이전트 간 도구 안전하게 공유
- **Lab 4:** 프로덕션 배포 - AgentCore Runtime으로 관측성 확보
- **Lab 5:** 사용자 인터페이스 구축 - 고객 대상 애플리케이션 생성

### AgentCore Gateway & 도구 공유가 중요한 이유

현재 상태 (Lab 1-2): 각 에이전트가 자체 도구 사본을 가지고 있습니다. 실제로는 확장 가능하지 않고 다음과 같은 문제를 야기합니다:

- 다른 에이전트 간 코드 중복
- 일관성 없는 도구 동작 및 유지 관리 오버헤드
- 중앙화된 보안 또는 접근 제어 없음
- 여러 사용 사례로 확장하기 어려움

이 랩 후에는 다음을 제공할 수 있는 중앙화된 재사용 가능한 도구를 갖게 됩니다:

- 이커머스 고객 지원 에이전트 (현재 사용 사례)
- 판매 에이전트 (동일한 상품 정보 및 고객 데이터 필요)
- 재고 에이전트 (동일한 상품 정보 및 보증 확인 필요)
- 반품 처리 에이전트 (반품 정책 및 고객 프로필 필요)

그리고 기타 사용 사례들.

### AgentCore Identity로 보안 인증 추가

또한 AgentCore Gateway는 인바운드 및 아웃바운드 연결을 안전하게 인증해야 합니다. [AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html)는 Okta, Entra, Amazon Cognito와 같은 표준 ID 공급자를 지원하면서 AWS 서비스 및 Slack, Zoom과 같은 타사 애플리케이션 전반에 걸쳐 원활한 에이전트 ID 및 액세스 관리를 제공합니다. 이 랩에서는 AgentCore Gateway가 AgentCore Identity와 통합하여 인바운드 및 아웃바운드 인증을 통해 보안 연결을 제공하는 방법을 살펴보겠습니다.

인바운드 인증의 경우 AgentCore Gateway는 호출 중에 전달된 OAuth 토큰을 분석하여 게이트웨이의 도구에 대한 액세스를 허용하거나 거부할지 결정합니다. 도구가 외부 리소스에 액세스해야 하는 경우 AgentCore Gateway는 API 키, IAM 또는 OAuth 토큰을 통해 아웃바운드 인증을 사용하여 외부 리소스에 대한 액세스를 허용하거나 거부할 수 있습니다.

## Lab 3을 위한 아키텍처

<div style="text-align:left">
    <img src="images/architecture_lab3_ecommerce_gateway.png" width="75%"/>
</div>

*반품 자격 검증 도구가 이제 보안 신원 기반 액세스 제어와 함께 AgentCore Gateway에서 중앙화되었습니다. 여러 에이전트와 사용 사례가 동일한 도구를 안전하게 공유할 수 있습니다. 기존 도구들(`process_return()`, `process_exchange()`, `web_search()`)은 이커머스 고객 지원 사용 사례에 특화되어 있으므로 로컬 도구로 유지됩니다.*

### 주요 기능
- **AWS Lambda 함수 원활하게 통합:** 이 예제는 기존 AWS Lambda 함수와 에이전트를 통합하여 Amazon Bedrock AgentCore Gateway를 사용하여 항목의 반품 자격을 확인하는 방법을 보여줍니다.
- **인바운드 인증으로 Gateway 엔드포인트 보안:** 유효한 JWT 토큰을 제공하는 에이전트만 엔드포인트에 연결하여 도구를 사용할 수 있습니다
- **MCP 엔드포인트를 사용하도록 에이전트 구성:** 에이전트가 유효한 JWT 토큰을 가져와서 AgentCore Gateway에서 제공하는 MCP 엔드포인트에 연결하는 데 사용합니다

## 전제 조건

* Python 3.10+
* AWS 자격 증명 구성
* Anthropic Claude 3.7 [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 활성화
* Lab 2 이커머스 고객 지원 에이전트에 메모리 추가 완료
* 이러한 리소스는 AWS 워크숍 계정 내에서 생성됩니다
    - AWS Lambda 함수
    - AWS Lambda 실행 IAM 역할
    - AgentCore Gateway IAM 역할
    - AWS Lambda 함수에서 사용하는 DynamoDB 테이블
    - Cognito 사용자 풀 및 사용자 풀 클라이언트

#### ⚠️ 중요: 노트북 재시작 필요

**만약 이전에 이 노트북을 실행했다면:**
1. **Kernel > Restart Kernel** 을 클릭하여 노트북을 재시작하세요
2. 그 다음 위의 인프라 배포 셀부터 다시 실행하세요

이는 SSM 파라미터 경로가 `/app/customersupport` → `/app/ecommerce`로 변경되었기 때문입니다.

---

#### 🏠 자율 학습 사용자 주의사항

**위의 인프라 배포 셀을 이미 실행했다면 이 셀은 건너뛰세요.**

자율 학습으로 실행하는 경우에만 아래 주석을 해제하고 실행하세요:

# ⚠️ 자율 학습 사용자만 실행 (위에서 인프라 배포를 이미 했다면 건너뛰기)
# !bash ../../../scripts/prereq.sh

In [1]:
# AWS 인프라 자동 배포 스크립트 실행
import subprocess
import os
import sys
from pathlib import Path

print("🚀 AWS 인프라 배포를 시작합니다...")
print("=" * 60)

# 프로젝트 루트로 이동
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
os.chdir(project_root)

print(f"📂 작업 디렉토리: {os.getcwd()}")
print("⏳ CloudFormation 스택 배포 중... (3-5분 소요)")
print("💡 배포 진행상황을 확인하려면 AWS Console > CloudFormation에서 확인하세요.")
print()

try:
    # prereq.sh 스크립트 실행
    result = subprocess.run(
        ["./scripts/prereq.sh"], 
        capture_output=True, 
        text=True, 
        timeout=600  # 10분 타임아웃
    )
    
    print("📄 배포 결과:")
    print("-" * 40)
    print(result.stdout)
    
    if result.stderr:
        print("⚠️ 경고/오류:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("✅ AWS 인프라 배포가 성공적으로 완료되었습니다!")
        print()
        print("🎯 생성된 리소스:")
        print("   • S3 버킷: Lambda 코드 저장")
        print("   • Lambda 함수: 반품 자격 검증 API")
        print("   • Cognito User Pool: OAuth 인증")
        print("   • DynamoDB 테이블: 고객 데이터")
        print("   • SSM 파라미터: 구성 정보")
        print()
        print("🚀 이제 아래 셀들을 순서대로 실행하세요!")
    else:
        print("❌ 인프라 배포에 실패했습니다.")
        print("💡 AWS 자격 증명과 권한을 확인해주세요.")
        
except subprocess.TimeoutExpired:
    print("⏰ 배포 시간이 초과되었습니다. AWS Console에서 스택 상태를 확인해주세요.")
except Exception as e:
    print(f"❌ 배포 중 오류가 발생했습니다: {str(e)}")
    print("💡 scripts/prereq.sh 파일이 존재하는지 확인해주세요.")

print("=" * 60)

🚀 AWS 인프라 배포를 시작합니다...
📂 작업 디렉토리: /home/ubuntu/Self-Study-Generative-AI/lab/18_ec-customer-support-agent-bedrock_agent_core
⏳ CloudFormation 스택 배포 중... (3-5분 소요)
💡 배포 진행상황을 확인하려면 AWS Console > CloudFormation에서 확인하세요.



📄 배포 결과:
----------------------------------------
Region: us-east-1
Account ID: 057716757052
🪣 Using S3 bucket: ecommercesupport112-057716757052-us-east-1
{
    "Location": "/ecommercesupport112-057716757052-us-east-1"
}
Reading package lists...
Building dependency tree...
Reading state information...
zip is already the newest version (3.0-12build2).
The following package was automatically installed and is no longer required:
  linux-headers-aws
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
📦 Zipping contents of prerequisite/lambda/python into lambda.zip...
☁️ Uploading lambda.zip to s3://ecommercesupport112-057716757052-us-east-1/lambda.zip...
Completed 1.0 MiB/8.2 MiB (12.3 MiB/s) with 1 file(s) remaining
Completed 2.0 MiB/8.2 MiB (20.9 MiB/s) with 1 file(s) remaining
Completed 3.0 MiB/8.2 MiB (30.1 MiB/s) with 1 file(s) remaining
Completed 4.0 MiB/8.2 MiB (38.8 MiB/s) with 1 file(s) remaining
Completed 5.0 MiB/8.2 MiB (46.8 Mi

### 📋 배포 상태 확인 (선택사항)

AWS 인프라가 제대로 배포되었는지 확인하려면 아래 셀을 실행하세요:

In [2]:
# 배포된 AWS 리소스 확인
import boto3
import json
from datetime import datetime

print("🔍 배포된 AWS 리소스 확인 중...")
print("=" * 50)

try:
    # CloudFormation 스택 상태 확인
    cf_client = boto3.client('cloudformation')
    
    stacks = [
        'EcommerceCustomerSupportStackInfra',
        'EcommerceCustomerSupportStackCognito'
    ]
    
    print("📚 CloudFormation 스택 상태:")
    for stack_name in stacks:
        try:
            response = cf_client.describe_stacks(StackName=stack_name)
            stack = response['Stacks'][0]
            status = stack['StackStatus']
            created = stack['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
            
            status_emoji = "✅" if "COMPLETE" in status else "⏳" if "PROGRESS" in status else "❌"
            print(f"   {status_emoji} {stack_name}: {status} (생성: {created})")
        except:
            print(f"   ❌ {stack_name}: 스택을 찾을 수 없습니다")
    
    print()
    
    # SSM 파라미터 확인
    ssm_client = boto3.client('ssm')
    try:
        response = ssm_client.get_parameters_by_path(
            Path='/app/ecommerce/agentcore',
            Recursive=True
        )
        
        print(f"📋 SSM 파라미터 ({len(response['Parameters'])}개):")
        for param in sorted(response['Parameters'], key=lambda x: x['Name']):
            name = param['Name'].split('/')[-1]
            print(f"   ✅ {name}")
        print()
        
    except Exception as e:
        print(f"   ❌ SSM 파라미터 조회 실패: {str(e)}")
    
    # Lambda 함수 확인
    lambda_client = boto3.client('lambda')
    try:
        functions = lambda_client.list_functions()['Functions']
        ecommerce_functions = [f for f in functions if 'EcommerceCustomer' in f['FunctionName']]
        
        print(f"🔧 Lambda 함수 ({len(ecommerce_functions)}개):")
        for func in ecommerce_functions:
            print(f"   ✅ {func['FunctionName']} ({func['Runtime']})")
        print()
        
    except Exception as e:
        print(f"   ❌ Lambda 함수 조회 실패: {str(e)}")
    
    print("🎯 준비 완료! 이제 Step 1부터 진행하세요.")
    
except Exception as e:
    print(f"❌ 리소스 확인 중 오류: {str(e)}")
    print("💡 AWS 자격 증명을 확인하고 인프라 배포를 다시 시도해주세요.")

print("=" * 50)

🔍 배포된 AWS 리소스 확인 중...
📚 CloudFormation 스택 상태:
   ✅ EcommerceCustomerSupportStackInfra: CREATE_COMPLETE (생성: 2025-08-17 02:57:31)
   ✅ EcommerceCustomerSupportStackCognito: CREATE_COMPLETE (생성: 2025-08-17 02:59:08)

📋 SSM 파라미터 (10개):
   ✅ cognito_auth_scope
   ✅ cognito_auth_url
   ✅ cognito_discovery_url
   ✅ cognito_domain
   ✅ cognito_token_url
   ✅ gateway_iam_role
   ✅ lambda_arn
   ✅ memory_id
   ✅ runtime_iam_role
   ✅ userpool_id

🔧 Lambda 함수 (2개):
   ✅ EcommerceCustomerSupportStac-CustomerSupportLambda-E7Jh2ObVr3Oy (python3.12)
   ✅ EcommerceCustomerSupportStackCo-PostSignupFunction-79ldVH4D8Vmm (python3.13)

🎯 준비 완료! 이제 Step 1부터 진행하세요.


## Step 1: 필요한 라이브러리 설치 및 가져오기

In [3]:
# 라이브러리 가져오기
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
import os
import sys
import boto3
import json
from bedrock_agentcore.identity.auth import requires_access_token
from mcp.client.streamable_http import streamablehttp_client
import requests

# 프로젝트 루트 경로를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from lab_helpers.utils import get_ssm_parameter, put_ssm_parameter, load_api_spec, get_cognito_client_secret

sts_client = boto3.client('sts')

# AWS 계정 세부 정보 가져오기
REGION = boto3.session.Session().region_name

gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=REGION,
)

print("✅ 라이브러리 가져오기 성공!")
print(f"📂 프로젝트 루트: {project_root}")

✅ 라이브러리 가져오기 성공!
📂 프로젝트 루트: /home/ubuntu


## Step 2: 에이전트에 기존 이커머스 데이터에 액세스할 수 있는 도구 제공
AgentCore Gateway는 세 가지 주요 방법으로 에이전트 도구 통합을 단순화합니다:

범용 MCP 지원: AgentCore Gateway의 MCP 표준을 통해 노출하여 도구를 모든 에이전트 프레임워크와 즉시 호환되도록 만듭니다

간단한 REST 통합: 기존 REST 서비스를 AgentCore Gateway 대상으로 추가하기만 하면 에이전트 도구로 변환합니다

Lambda 유연성: 모든 API를 호출할 수 있는 MCP 엔드포인트로 Lambda 함수를 노출합니다 - 여기서는 반품 자격 상태를 확인하는 함수로 시연합니다

AgentCore Gateway는 호출할 도구의 이름으로 Lambda 컨텍스트를 채우고, 도구에 전달된 매개변수는 Lambda 이벤트에서 제공됩니다:

```
extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
resource = extended_tool_name.split("___")[1]
```

[Lambda 함수](./prerequisite/lambda/return_eligibility_check.py)

```
def lambda_handler(event, context):
    if get_tool_name(event) == "check_return_eligibility":
        order_number = get_named_parameter(event=event, name="order_number")
        customer_id = get_named_parameter(event=event, name="customer_id")

        eligibility_result = check_return_eligibility(order_number, customer_id)
        return {"statusCode": 200, "body": eligibility_result}
```

## Step 3: 도구를 MCP로 변환
이제 AgentCore Gateway를 사용하여 MCP 서버를 개발하고 있으므로 여러 에이전트에 사용할 것으로 생각되는 모든 도구를 MCP화할 수 있습니다. 이러한 도구 중 하나는 Lab1에서 구축한 반품 자격 검증 도구일 수 있습니다. 결과적으로 우리는 Lab 1의 반품 자격 검증 도구를 AgentCore Gateway 내의 Lambda 도구로도 변환했습니다:

[반품 자격 검증 Lambda](./prerequisite/lambda/return_eligibility_check.py)
```
def check_return_eligibility(order_number: str, customer_id: str) -> Dict[str, Any]:
    """패션/뷰티 상품의 반품 자격을 자동으로 검증합니다."""
    # 실제 환경에서는 DynamoDB나 RDS에서 조회
    # VIP 등급, 배송일, 상품 상태 등을 종합 판단
    return eligibility_result
```

## Step 4: 함수 정의 메타데이터 생성
마지막으로 Lambda 함수에서 구현한 도구를 설명하는 도구 스키마를 작성해야 합니다.

이 파일은 이미 [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json)에 정의되어 있습니다

```
[{
    "name": "check_return_eligibility",
    "description": "패션/뷰티 상품의 반품 자격을 자동으로 검증합니다. 주문일, 배송일, 상품 상태, VIP 등급 등을 종합적으로 판단하여 반품 가능 여부를 결정합니다.",
    "inputSchema": {
        "type": "object",
        "properties": {
            "order_number": {
                "type": "string",
                "description": "주문번호 (예: 'KS-2024-001234')"
            },
            "customer_id": {
                "type": "string", 
                "description": "고객 ID (예: 'customer_ecommerce_001')"
            }
        },
        "required": ["order_number", "customer_id"]
    }
}]
```

## Step 5. AgentCore Gateway 생성

이제 Lambda 함수를 MCP 호환 엔드포인트로 노출하는 AgentCore Gateway를 생성해보겠습니다.

호출자가 도구를 호출할 권한이 있는지 확인하기 위해 인바운드 인증을 구성해야 합니다.

인바운드 인증은 MCP 서버의 표준인 OAuth 인증을 사용하여 작동합니다. OAuth를 사용하면 클라이언트 애플리케이션이 Gateway를 사용하기 전에 OAuth 인증자로 인증해야 합니다. 클라이언트는 런타임에 사용되는 액세스 토큰을 받습니다.

OAuth 검색 서버와 클라이언트 ID를 지정해야 합니다. 워크숍과 함께 제공된 Cloudformation은 이미 Cognito UserPool과 UserPoolClient를 프로비저닝했으며 전용 SSM 매개변수에 검색 URL과 클라이언트 ID를 저장했습니다.

In [4]:
gateway_name = "ecommerce-gw"

# 이커머스 전용 인증 설정
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            get_ssm_parameter("/app/ecommerce/agentcore/machine_client_id")
        ],
        "discoveryUrl": get_ssm_parameter("/app/ecommerce/agentcore/cognito_discovery_url")
    }
}

try:
    # 새 게이트웨이 생성
    print(f"리전 {REGION}에서 이름이 {gateway_name}인 게이트웨이 생성 중")

    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=get_ssm_parameter("/app/ecommerce/agentcore/gateway_iam_role"),
        protocolType="MCP",
        authorizerType="CUSTOM_JWT",
        authorizerConfiguration=auth_config,
        description="이커머스 고객 지원 AgentCore Gateway",
    )

    gateway_id = create_response["gatewayId"]

    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    put_ssm_parameter("/app/ecommerce/agentcore/gateway_id", gateway_id)

    print(f"✅ 게이트웨이가 ID {gateway_id}로 성공적으로 생성되었습니다")

except Exception as e:
    # 게이트웨이가 존재하는 경우 SSM에서 기존 게이트웨이 ID 수집
    try:
        existing_gateway_id = get_ssm_parameter("/app/ecommerce/agentcore/gateway_id")
        print(f"ID {existing_gateway_id}인 기존 게이트웨이를 찾았습니다")
        
        # 기존 게이트웨이 세부 정보 가져오기
        gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)
        gateway = {
            "id": existing_gateway_id,
            "name": gateway_response["name"],
            "gateway_url": gateway_response["gatewayUrl"],
            "gateway_arn": gateway_response["gatewayArn"],
        }
        gateway_id = gateway['id']
    except Exception as inner_e:
        print(f"❌ 게이트웨이 생성/조회 실패: {str(e)}")
        print(f"   추가 오류: {str(inner_e)}")
        raise

리전 us-east-1에서 이름이 ecommerce-gw인 게이트웨이 생성 중
✅ 게이트웨이가 ID ecommerce-gw-9i0zzbzrfi로 성공적으로 생성되었습니다


## Step 6. Lambda 함수 대상 추가
이제 이전에 정의된 함수 정의를 [prerequisite/lambda/api_spec.json](./prerequisite/lambda/api_spec.json)에서 사용하여 Agent Gateway 내에 Lambda 대상을 생성하겠습니다. 이렇게 하면 게이트웨이가 호스팅할 도구가 정의됩니다.

Gateway를 사용하면 Gateway에 여러 대상을 연결할 수 있으며 언제든지 게이트웨이에 연결된 대상/도구를 변경할 수 있습니다. 각 대상은 자체 자격 증명 공급자를 가질 수 있지만 Gateway는 무수한 API에서 에이전트에 대한 모든 관련 도구에 액세스할 수 있게 하는 단일 MCP URL이 됩니다.

In [5]:
def load_api_spec(file_path: str) -> list:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    if not isinstance(data, list):
        raise ValueError("JSON 파일에 리스트가 예상됩니다")
    return data

try:
    # API 스펙 파일 경로 후보들
    api_spec_candidates = [
        "./prerequisite/lambda/api_spec.json",  # 노트북 로컬 경로
        os.path.join(project_root, "prerequisite/lambda/api_spec.json"),  # 프로젝트 루트 경로
    ]
    
    api_spec_file = None
    for candidate in api_spec_candidates:
        if os.path.exists(candidate):
            api_spec_file = candidate
            break
    
    if api_spec_file is None:
        print(f"❌ API 명세 파일을 다음 경로에서 찾을 수 없습니다:")
        for candidate in api_spec_candidates:
            print(f"   - {candidate}")
        sys.exit(1)

    api_spec = load_api_spec(api_spec_file)
    print(f"✅ API 명세 파일 로드 완료: {api_spec_file}")
 
    # Gateway에 인바운드 OAuth를 위해 Cognito 사용
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": get_ssm_parameter("/app/ecommerce/agentcore/lambda_arn"),
                "toolSchema": {"inlinePayload": api_spec},
            }
        }
    }

    # 게이트웨이 대상 생성
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="EcommerceLambdaTarget",
        description="이커머스 반품 자격 검증 Lambda 대상",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )

    print(f"✅ 게이트웨이 대상 생성됨: {create_target_response['targetId']}")

except Exception as e:
    print(f"❌ 게이트웨이 대상 생성 오류: {str(e)}")

✅ API 명세 파일 로드 완료: ./prerequisite/lambda/api_spec.json
✅ 게이트웨이 대상 생성됨: XVJK5P2QHK


## Step 7: 새로운 MCP 기반 도구를 지원 에이전트에 추가
여기서 Cognito의 인증 토큰을 Strands SDK의 MCPClient에 통합하여 Strands Agent와 통합할 MCP Server 객체를 생성합니다

In [6]:
def get_token(client_id: str, client_secret: str, scope_string: str, url: str) -> dict:
    try:
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        data = {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret,
            "scope": scope_string,
        }
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as err:
        return {"error": str(err)}

## Step 7.1. 보안 MCP 클라이언트 객체 설정

In [7]:
client_id = get_ssm_parameter("/app/ecommerce/agentcore/machine_client_id")
client_secret = get_cognito_client_secret()
scope = get_ssm_parameter("/app/ecommerce/agentcore/cognito_auth_scope")
token_url = get_ssm_parameter("/app/ecommerce/agentcore/cognito_token_url")

print("클라이언트 ID: ", client_id)
print("클라이언트 시크릿: ", client_secret[:10] + "...")
print("스코프: ", scope)
print("토큰 URL: ", token_url)

print(f"\nGateway 엔드포인트 - MCP URL: {gateway['gateway_url']}")

# 토큰 요청
gateway_access_token = get_token(
    client_id,
    client_secret,
    scope,
    token_url
)

print(f"토큰 상태: {'성공' if 'access_token' in gateway_access_token else '실패'}")

# MCP 클라이언트 설정
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway['gateway_url'],
        headers={"Authorization": f"Bearer {gateway_access_token['access_token']}"},
    )
)

print("✅ MCP 클라이언트 설정 완료")

클라이언트 ID:  1tu3b7qdh20ihfiocvoki2a6es
클라이언트 시크릿:  gs3dl63o80...
스코프:  default-m2m-resource-server-24babc40/read
토큰 URL:  https://us-east-124babc40.auth.us-east-1.amazoncognito.com/oauth2/token

Gateway 엔드포인트 - MCP URL: https://ecommerce-gw-9i0zzbzrfi.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp
토큰 상태: 성공
✅ MCP 클라이언트 설정 완료


## Step 7.2. MCP 클라이언트를 사용하여 에이전트에서 도구 액세스
이제 구축한 AgentCore Gateway와 이전 랩의 리소스를 함께 사용하여 Strands Agent를 생성하겠습니다. 우리 에이전트는 이제 Strands Agent를 통한 로컬 도구와 AgentCore Gateway를 통한 MCP 도구의 혼합을 사용합니다

In [8]:
# Lab 1과 Lab 2에서 생성한 도구와 메모리 가져오기
try:
    # 새로운 구조에서 import 시도
    from use_cases.customer_support.agent import (
        SYSTEM_PROMPT,
        process_return,
        process_exchange, 
        web_search,
        MODEL_ID
    )
    print("✅ customer support agent 모듈 import 완료")
except ImportError:
    try:
        # legacy 경로에서 import 시도
        from legacy.original_files.ecommerce_agent import (
            SYSTEM_PROMPT,
            process_return,
            process_exchange, 
            web_search,
            MODEL_ID
        )
        print("✅ legacy ecommerce_agent 모듈 import 완료")
    except ImportError:
        # 노트북 로컬 copy에서 import 시도
        from ecommerce_agent import (
            SYSTEM_PROMPT,
            process_return,
            process_exchange, 
            web_search,
            MODEL_ID
        )
        print("✅ 로컬 ecommerce_agent 모듈 import 완료")

from lab_helpers.ecommerce_memory import (
    EcommerceCustomerMemoryHooks,
    create_or_get_ecommerce_memory_resource
)
import uuid
from bedrock_agentcore.memory import MemoryClient

memory_client = MemoryClient(region_name=REGION)

memory_id = create_or_get_ecommerce_memory_resource()
SESSION_ID = str(uuid.uuid4())
CUSTOMER_ID = "customer_ecommerce_001"
memory_hooks = EcommerceCustomerMemoryHooks(memory_id, memory_client, CUSTOMER_ID, SESSION_ID)

# Bedrock 모델 초기화
model = BedrockModel(
    model_id=MODEL_ID,
    temperature=0.3,  # 창의성과 일관성 사이의 균형
    region_name=REGION
)

# MCP 클라이언트 초기화 시도 (오류 처리 포함)
try:
    mcp_client.start()
    mcp_tools = mcp_client.list_tools_sync()
    print(f"✅ MCP 클라이언트 연결 성공! 사용 가능한 도구: {len(mcp_tools)}개")
except Exception as e:
    print(f"⚠️ MCP 클라이언트 연결 실패: {str(e)}")
    print("💡 로컬 도구만 사용하여 에이전트를 생성합니다.")
    mcp_tools = []

# 로컬 도구와 MCP 도구 결합
tools = [
    process_return,      # 로컬 도구: 반품 처리
    process_exchange,    # 로컬 도구: 교환 처리
    web_search,         # 로컬 도구: 웹 검색
] + mcp_tools

print(f"📋 총 도구 개수: {len(tools)}개")
print("   • 반품 처리 (process_return)")
print("   • 교환 처리 (process_exchange)")
print("   • 웹 검색 (web_search)")
if mcp_tools:
    print(f"   • MCP 도구: {len(mcp_tools)}개")

# 이커머스 고객 지원 에이전트 생성
agent = Agent(
    model=model,
    tools=tools,
    hooks=[memory_hooks],
    system_prompt=SYSTEM_PROMPT
)

print("✅ 이커머스 고객 지원 에이전트가 성공적으로 생성되었습니다!")
print("🔧 현재 구성: 로컬 도구 3개" + (f" + MCP 도구 {len(mcp_tools)}개" if mcp_tools else " (MCP 연결 실패)"))

✅ customer support agent 모듈 import 완료


✅ MCP 클라이언트 연결 성공! 사용 가능한 도구: 1개
📋 총 도구 개수: 4개
   • 반품 처리 (process_return)
   • 교환 처리 (process_exchange)
   • 웹 검색 (web_search)
   • MCP 도구: 1개
✅ 이커머스 고객 지원 에이전트가 성공적으로 생성되었습니다!
🔧 현재 구성: 로컬 도구 3개 + MCP 도구 1개


## Step 8: MCP 도구 액세스로 에이전트 테스트"

모든 기능이 올바르게 작동하는지 확인하기 위해 샘플 쿼리로 에이전트를 테스트해보겠습니다.

In [9]:
test_prompts = [
    # 도구 목록 확인
    "사용 가능한 모든 도구를 나열해주세요",
    
    # 반품 처리 (로컬 도구)
    "지난주 산 플라워 패턴 원피스 사이즈가 작아서 반품하고 싶어요. 주문번호는 KS-2024-001234예요.",
    
    # 스타일링 조언 (로컬 도구) 
    "겨울 코트 스타일링 방법을 알려주세요",
    
    # 교환 처리 (로컬 도구)
    "청바지를 M에서 L로 교환하고 싶어요"
]

# 에이전트 테스트 함수
def test_agent_responses(agent, prompts):
    for i, prompt in enumerate(prompts, 1):
        print(f"\n테스트 케이스 {i}: {prompt}")
        print("-" * 50)
        try:
            response = agent(prompt)
            print(f"응답: {response}")
        except Exception as e:
            print(f"❌ 오류: {str(e)}")
        print("-" * 50)

# 테스트 실행
print("🧪 이커머스 고객 지원 에이전트 테스트 시작")
print("=" * 60)

test_agent_responses(agent, test_prompts)

print("\n✅ 기본 테스트 완료!")
print("\n💡 참고:")
print("   • 현재 로컬 도구 3개 (반품, 교환, 웹검색)가 정상 작동하고 있습니다.")
if mcp_tools:
    print(f"   • MCP Gateway 도구 {len(mcp_tools)}개도 사용 가능합니다.")
else:
    print("   • MCP Gateway 연결이 실패했지만, 로컬 도구로 완전한 고객 지원이 가능합니다.")
print("   • Lab 4에서 프로덕션 환경에서의 Gateway 통합을 완성하겠습니다.")

🧪 이커머스 고객 지원 에이전트 테스트 시작

테스트 케이스 1: 사용 가능한 모든 도구를 나열해주세요
--------------------------------------------------
안녕하세요, K-Style 쇼핑몰 고객님! 저희 쇼핑몰에서 제공해 드릴 수 있는 도구들을 안내해 드리겠습니다.

현재 사용 가능한 도구는 다음과 같습니다:

1. **반품 처리 서비스 (process_return)**
   - 주문번호, 상품명, 반품 사유(사이즈, 색상, 품질, 변심 등)를 입력하시면 신속하게 반품 처리를 도와드립니다.

2. **교환 처리 서비스 (process_exchange)**
   - 주문번호, 상품명, 현재 옵션(예: "화이트/M"), 원하는 옵션(예: "블랙/L")을 입력하시면 빠른 교환 서비스를 제공해 드립니다.

3. **패션/뷰티 정보 검색 서비스 (web_search)**
   - 패션, 뷰티, 트렌드, 케어 등에 관한 정보를 검색하여 전문적인 조언을 제공해 드립니다.

4. **반품 자격 확인 서비스**
   - 고객 ID와 주문번호를 통해 반품 가능 여부를 자동으로 확인해 드립니다.

고객님께서는 건성 피부에 맞는 보습 효과가 있는 파운데이션이나, 사이즈가 작게 나오는 브랜드의 의류에 관심이 있으신 것으로 보입니다. 어떤 도구를 이용하여 도움을 드릴까요? 특정 상품에 대한 반품이나 교환을 원하시거나, 패션/뷰티 관련 정보가 필요하시면 말씀해 주세요!응답: 안녕하세요, K-Style 쇼핑몰 고객님! 저희 쇼핑몰에서 제공해 드릴 수 있는 도구들을 안내해 드리겠습니다.

현재 사용 가능한 도구는 다음과 같습니다:

1. **반품 처리 서비스 (process_return)**
   - 주문번호, 상품명, 반품 사유(사이즈, 색상, 품질, 변심 등)를 입력하시면 신속하게 반품 처리를 도와드립니다.

2. **교환 처리 서비스 (process_exchange)**
   - 주문번호, 상품명, 현재 옵션(예: "화이트/M"), 원하는 옵션(

### 축하합니다! 🎉

**Lab 3: AgentCore Gateway로 도구를 에이전트에 안전하게 연결**을 성공적으로 완료했습니다

달성한 것:

##### 도구 중앙화 및 재사용성:

- 반품 자격 검증을 로컬 도구에서 중앙화된 AgentCore Gateway로 마이그레이션
- 기존 엔터프라이즈 Lambda 함수 통합 (반품 자격 검증)
- 여러 에이전트 유형이 액세스할 수 있는 공유 도구 인프라 생성

##### 엔터프라이즈급 보안:

- Cognito 통합으로 JWT 기반 인증 구현
- 게이트웨이 액세스를 위한 보안 인바운드 인증 구성
- 도구 사용을 위한 신원 기반 액세스 제어 설정

##### 확장 가능한 아키텍처 기반:

- 여러 사용 사례를 제공하는 재사용 가능한 도구 구축 (이커머스 고객 지원, 판매, 반품 처리)
- 다른 에이전트 간 코드 중복 제거
- 도구 업데이트 및 유지 관리를 위한 중앙화된 관리 생성

##### 이커머스 특화 성과:

- **자동 반품 자격 검증**: VIP 등급, 배송일, 상품 상태 종합 판단
- **패션/뷰티 카테고리별 정책**: 각 제품군의 특성을 반영한 차별화된 반품 조건
- **고객 등급별 혜택**: VIP 고객 추가 반품 기간 제공
- **한국어 맞춤 응답**: 친근하고 정중한 고객 응대

##### 현재 한계점 (다음에 해결 예정!):

- **로컬 개발 환경** - 아직 노트북에서 실행, 프로덕션 준비 안됨
- **제한된 관측성** - 에이전트 동작 및 성능에 대한 포괄적인 모니터링 없음
- **수동 확장** - 증가된 부하나 여러 동시 사용자를 자동으로 처리할 수 없음

##### 다음 단계: Lab 4 - AgentCore Runtime으로 프로덕션 배포

Lab 4에서는 프로토타입을 다음과 같은 프로덕션 준비 시스템으로 변환하겠습니다:

- 확장 가능한 에이전트 배포를 위한 AgentCore Runtime
- 메트릭, 로깅 및 추적을 통한 포괄적인 관측성
- 실제 트래픽을 처리하는 자동 확장 기능

### 리소스
- [Amazon Bedrock Agent Core Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html)
- [Strands Agents 문서](https://github.com/strands-agents/sdk-python)
- [공식 이커머스 고객 지원 샘플](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/02-use-cases/customer-support-assistant)